# Проект для «Викишоп»

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Нужно обучить модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Значение метрики качества *F1* должно быть не меньше 0.75. 

Данные находятся в файле `toxic_comments.csv`. Столбец `text` в нём содержит текст комментария, а `toxic` — целевой признак.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Импорт-библиотек-и-загрузка-данных" data-toc-modified-id="Импорт-библиотек-и-загрузка-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Импорт библиотек и загрузка данных</a></span></li><li><span><a href="#Просмотр-информации-о-данных-и-удаление-неинформативного-столбца" data-toc-modified-id="Просмотр-информации-о-данных-и-удаление-неинформативного-столбца-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Просмотр информации о данных и удаление неинформативного столбца</a></span></li><li><span><a href="#Предобработка-текста" data-toc-modified-id="Предобработка-текста-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Предобработка текста</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Разделение-на-выборки" data-toc-modified-id="Разделение-на-выборки-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Разделение на выборки</a></span></li><li><span><a href="#Обучение-моделей" data-toc-modified-id="Обучение-моделей-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Обучение моделей</a></span><ul class="toc-item"><li><span><a href="#LogisticRegression()" data-toc-modified-id="LogisticRegression()-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span><code>LogisticRegression()</code></a></span></li><li><span><a href="#DecisionTreeClassifier()" data-toc-modified-id="DecisionTreeClassifier()-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span><code>DecisionTreeClassifier()</code></a></span></li><li><span><a href="#RandomForestClassifier()" data-toc-modified-id="RandomForestClassifier()-2.2.3"><span class="toc-item-num">2.2.3&nbsp;&nbsp;</span><code>RandomForestClassifier()</code></a></span></li></ul></li><li><span><a href="#Тестирование-лучшей-модели" data-toc-modified-id="Тестирование-лучшей-модели-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Тестирование лучшей модели</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

## Подготовка

### Импорт библиотек и загрузка данных

Для начала импортируем все необходимые библиотеки и инструменты.

In [1]:
%pip install spacy
!spacy download en_core_web_sm

  Using cached colorama-0.4.6-py2.py3-none-any.whl (25 kB)
  Attempting uninstall: colorama
    Found existing installation: colorama 0.4.4
    Uninstalling colorama-0.4.4:
      Successfully uninstalled colorama-0.4.4
  Attempting uninstall: smart-open
    Found existing installation: smart-open 5.1.0
    Uninstalling smart-open-5.1.0:
      Successfully uninstalled smart-open-5.1.0
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [4]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import nltk
import re

from tqdm.notebook import tqdm
tqdm.pandas()

from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import spacy

Зафиксируем случайное состояние в константе.

In [5]:
RS = 322

Загрузим данные в переменную `data`, выведем основную информацию о них методом `info()` а также посмотрим первые строки.

In [6]:
data = pd.read_csv('toxic_comments.csv')

### Просмотр информации о данных и удаление неинформативного столбца

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [8]:
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


В данных есть непонятный столбец `'Unnamed: 0'`, который совпадает с индексами. Выведем последние значения этого столбца.

In [9]:
data['Unnamed: 0'].tail()

159287    159446
159288    159447
159289    159448
159290    159449
159291    159450
Name: Unnamed: 0, dtype: int64

В конце значения столбца перестают совпадать с индексами. Проверим монотонный ли столбец, и посмотрим, есть ли в нём дубликаты, чтобы проверить имеет ли он какое-то значение для нас.

In [10]:
print(data['Unnamed: 0'].is_monotonic)
print(data['Unnamed: 0'].duplicated().sum())

True
0


Столбец монотонный и дубликатов нет. Выведем строки, где столбец не совпадает с индексом.

In [11]:
data.loc[data['Unnamed: 0'].index != data['Unnamed: 0']]

,Unnamed: 0,text,toxic
6080,6084,"""::I'll alos be looking in to see how this is ...",0
6081,6085,"""\n\nThe Ezekiel passage is quoted in the Molo...",0
6082,6086,Thank you for experimenting with Wikipedia. Y...,0
6083,6087,Any complaints with that as the new wording?,0
6084,6088,"""\nI also disagree with the merge as Strength ...",0
...,...,...,...
159287,159446,""":::::And for the second time of asking, when ...",0
159288,159447,You should be ashamed of yourself \n\nThat is ...,0
159289,159448,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,159449,And it looks like it was actually you who put ...,0


В какой-то момент, числа в этом столбце просто перестают совпадать с индексами, без какой-либо причины. Скорее всего данный столбец неинформативен и попал в данные по ошибке. Удалим его.

In [12]:
data.drop('Unnamed: 0', axis=1, inplace=True)

### Предобработка текста

Загрузим необходимые пакеты для библиотеки `nltk`. Также, загрузим список стоп-слов на английском языке в переменную `stopwords`.

In [13]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\acer\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Напишем функцию для очистки текста от спецсимволов и лемматизации. Для этого воспользуемся библиотекой `spacy`.

In [15]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def clear_lemmatize(text):
    cleaned_text = ' '.join(re.sub(r'[^a-zA-Z]', ' ', text).lower().split())
    doc = nlp(cleaned_text)
    return ' '.join([token.lemma_ for token in doc])

Применим к текстам функцию и сохраним результат в переменной `corpus`. 

In [16]:
corpus = data['text'].progress_apply(clear_lemmatize)

  0%|          | 0/159292 [00:00<?, ?it/s]

Сравним строки до и после преобразований:

In [18]:
data['text'].head()

0    Explanation\nWhy the edits made under my usern...
1    D'aww! He matches this background colour I'm s...
2    Hey man, I'm really not trying to edit war. It...
3    "\nMore\nI can't make any real suggestions on ...
4    You, sir, are my hero. Any chance you remember...
Name: text, dtype: object

In [19]:
corpus.head()

0    explanation why the edit make under my usernam...
1    d aww he match this background colour I m seem...
2    hey man I m really not try to edit war it s ju...
3    more I can t make any real suggestion on impro...
4    you sir be my hero any chance you remember wha...
Name: text, dtype: object

Лемматизация и очистка текста прошла успешно.

## Обучение

### Разделение на выборки

Создадим объект класса `TfidfVectorizer()` для преобразования данных, и разделим выборки на тренировочную, тестовую и валидационную в соотношении 3:1:1. Кросс-валидация в этом проекте не подойдёт, так как таким образом произойдёт утечка данных, потому что мы будем обучать "векторайзер" на всех тренировочных данных. 

In [20]:
tf_idf_vect = TfidfVectorizer(stop_words=stopwords)

In [21]:
train_features, test_features, train_target, test_target = train_test_split(corpus,
                                                                            data['toxic'],
                                                                            test_size=0.2,
                                                                            random_state=RS, 
                                                                            stratify=data['toxic'])
train_features, valid_features, train_target, valid_target = train_test_split(train_features,
                                                                             train_target, 
                                                                             test_size=0.25,
                                                                             random_state=RS,
                                                                             stratify=train_target)

Обучим векторайзер на тренировочных данных и сразу преобразуем их. Сохраним преобразованные данные в переменной `tf_idf_train`. 

In [22]:
tf_idf_train = tf_idf_vect.fit_transform(train_features)

Посмотрим на размер преобразованных данных.

In [23]:
tf_idf_train.shape

(95574, 111673)

После преобразования, у нас получилось 121456 уникальных слов без учёта стоп-слов. Преобразуем валидационную выборку.

In [24]:
tf_idf_valid = tf_idf_vect.transform(valid_features)

### Обучение моделей

Переберём гиперпараметры в цикле, так как нельзя использовать `GridSearchCV`. Попробуем модели логистической регрессии, дерева и случайного леса.

#### `LogisticRegression()`

Переберём в цикле гиперпараметр `C` для модели логистической регрессии. Проверим метрику F1 на валидационных данных.

In [25]:
%%time
best_lr_score = 0
for c in np.arange(6.0, 10.0, 1.0):
    model_lr = LogisticRegression(random_state=322, max_iter=150, class_weight='balanced', C=c)
    model_lr.fit(tf_idf_train, train_target)
    preds_lr = model_lr.predict(tf_idf_valid)
    score = f1_score(valid_target, preds_lr)
    if score > best_lr_score:
        best_lr_score = score
        best_lr_model = model_lr

CPU times: total: 1min 17s
Wall time: 29 s


In [26]:
print('Лучшая метрика F1 на линейной регрессии:', best_lr_score)
print('Гиперпараметр "C" на лучшей модели:', best_lr_model.get_params(deep=False)['C'])

Лучшая метрика F1 на линейной регрессии: 0.7692085081753726
Гиперпараметр "C" на лучшей модели: 9.0


#### `DecisionTreeClassifier()`

Для дерева переберём в цикле гиперпараметр `max_depth`.

In [27]:
%%time
best_tree_score = 0
score = None
for depth in range(5, 56, 10):
    model_tree = DecisionTreeClassifier(random_state=RS, max_depth=depth)
    model_tree.fit(tf_idf_train, train_target)
    preds_tree = model_tree.predict(tf_idf_valid)
    score = f1_score(valid_target, preds_tree)
    if score > best_tree_score:
        best_tree_score = score
        best_tree_model = model_tree

CPU times: total: 2min 4s
Wall time: 3min 16s


In [28]:
print('Лучшая метрика F1 на модели дерева:', best_tree_score)
print('Гиперпараметр "max_depth" на лучшей модели:', best_tree_model.get_params(deep=False)['max_depth'])

Лучшая метрика F1 на модели дерева: 0.7169744942832015
Гиперпараметр "max_depth" на лучшей модели: 55


#### `RandomForestClassifier()`

Для леса сделаем вложенный цикл и переберём гиперпараметры `max_depth` и `n_estimators`.

In [29]:
%%time
best_forest_score = 0
score = None
for depth in np.arange(5, 56, 10):
    for est in np.arange(5, 21, 5):
        model_forest = RandomForestClassifier(random_state=RS, max_depth=depth, n_estimators=est)
        model_forest.fit(tf_idf_train, train_target)
        preds_forest = model_forest.predict(tf_idf_valid)
        score = f1_score(valid_target, preds_forest)
        if score > best_forest_score:
            best_forest_score = score
            best_forest_model = model_forest

CPU times: total: 2min 16s
Wall time: 3min 44s


In [30]:
print('Лучшая метрика F1 на модели леса:', best_forest_score)
print('Гиперпараметр "max_depth" на лучшей модели:', best_forest_model.get_params(deep=False)['max_depth'])
print('Гиперпараметр "n_estimators" на лучшей модели:', best_forest_model.get_params(deep=False)['n_estimators'])

Лучшая метрика F1 на модели леса: 0.18354253835425383
Гиперпараметр "max_depth" на лучшей модели: 55
Гиперпараметр "n_estimators" на лучшей модели: 5


Модель случайного леса показала очень плохую метрику на валидационных данных. Второе место заняла модель дерева, однако метрика не достигает целевой. Лучшей оказалась модель логистической регрессии, она достигла целевой метрики F1 в 0.75. Проверим её на тестовых данных.

### Тестирование лучшей модели

Проверим лучшую модель логистической регрессии на тестовых данных.

In [31]:
tf_idf_test = tf_idf_vect.transform(test_features)
test_preds = best_lr_model.predict(tf_idf_test)
f1_score(test_target, test_preds)

0.7647653429602889

Целевая метрика достигнута. Задание выполнено.

## Выводы

* Лучшей моделью оказалась модель линейной регрессии. Она также оказалась самой быстрой. 
* Модель дерева показала среднюю метрику, немного не дотягивающую до целевой. 
* Модель случайного леса показала критически низкую метрику, а также работает дольше всех.